#Import DATASET

In [ ]:
import pandas as pd

df = pd.read_csv('/content/IMDB-Dataset.csv')

#Remove duplicate rows
df = df.drop_duplicates()

#Data Processing

In [ ]:
!pip install contractions

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

from bs4 import BeautifulSoup
import contractions

In [ ]:
stop = set(stopwords.words('english'))

In [ ]:
def expand_contractions(text):
    return contractions.fix(text)

In [ ]:
#Clean data function
def preprocess_text(text):
    wl =  WordNetLemmatizer()

    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    text = expand_contractions(text)

    emoji_clean = re. compile ("["
                                u"\U0001F600 -\U0001F64F " # emoticons
                                u"\U0001F300 -\U0001F5FF " # symbols & pictographs
                                u"\U0001F680 -\U0001F6FF " # transport & map symbols
                                u"\U0001F1E0 -\U0001F1FF " # flags (iOS)
                                u"\U00002702 -\U000027B0 "
                                u"\U000024C2 -\U0001F251 "
                                "]+", flags =re. UNICODE )
    text = emoji_clean.sub(r'', text)
    text = re.sub (r'\.(?=\ S)', '. ',text )
    text = re.sub (r'http \S+', '', text )
    text = ''.join(wl.lemmatize(word) for word in text.split() if word not in stop and word.isalpha())
    return text

In [ ]:
df['review'] = df['review'].apply(preprocess_text)

<ipython-input-25-b3f1a4bd6b7c>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


#Visualize Data

In [ ]:
df.head()

,review,sentiment
0,oneoftheotherreviewershasmentionedthatafterwat...,positive
1,awonderfullittleproductionthefilmingtechniquei...,positive
2,ithoughtthiswasawonderfulwaytospendtimeonatooh...,positive
3,asicallythereisafamilywherealittleboyjakethink...,negative
4,pettermatteiloveinthetimeofmoneyisavisuallystu...,positive


In [ ]:
freq_pos = len(df[df['sentiment'] == 'positive'])
freq_neg = len(df[df['sentiment'] == 'negative'])
print('Positive reviews: ', freq_pos)
print('Negative reviews: ', freq_neg)

Positive reviews:  24884
Negative reviews:  24698


In [ ]:
labels = ['positive', 'negative']

#Training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encode = LabelEncoder()
y_data = label_encode.fit_transform(df['sentiment'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['review'], y_data, test_size=0.2, random_state=42)

In [ ]:
#Vectorize data
tfidf_vectorizer = TfidfVectorizer ( max_features =10000)

x_train_encoded = tfidf_vectorizer.fit_transform (x_train )
x_test_encoded = tfidf_vectorizer.transform (x_test )

#Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
dt_classifier = DecisionTreeClassifier(criterion = 'entropy', random_state=42)
dt_classifier.fit(x_train_encoded, y_train)
y_pred = dt_classifier.predict(x_test_encoded)
accuracy = accuracy_score(y_pred, y_test)

0.5019663204598165

#Random Forest

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(x_train_encoded, y_train)
y_pred_rf = rf_classifier.predict(x_test_encoded)
accuracy_rf = accuracy_score(y_pred_rf, y_test)